# 衆議院議員選挙・小選挙区

データ出典: [衆議院議員選挙・小選挙区の統計データ及び地図データ（ポリゴンデータ）を提供するページ](https://gtfs-gis.jp/senkyoku/)


In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon

## ポリゴン


In [2]:
gdf_original = gpd.read_file(
    "./source/senkyoku2022/senkyoku2022.shp", encoding="shift-jis"
)
gdf_original

,UserID,ken,ku,kucode,kuname,geometry
0,1,1,8,108,北海道8区,"POLYGON ((140.96230 41.70940, 140.96221 41.709..."
1,2,1,8,108,北海道8区,"POLYGON ((140.96886 41.70992, 140.96871 41.709..."
2,3,1,8,108,北海道8区,"POLYGON ((140.97071 41.71056, 140.97067 41.710..."
3,4,1,8,108,北海道8区,"POLYGON ((140.97350 41.71136, 140.97339 41.711..."
4,5,1,8,108,北海道8区,"POLYGON ((140.98025 41.71242, 140.98018 41.712..."
...,...,...,...,...,...,...
119701,119706,47,3,4703,沖縄3区,"POLYGON ((127.94968 26.70849, 127.94977 26.708..."
119702,7963,1,4,104,北海道4区,"POLYGON ((140.51857 43.00028, 140.51911 43.000..."
119703,38493,13,21,1321,東京21区,"POLYGON ((139.38731 35.73909, 139.38754 35.739..."
119704,89067,40,2,4002,福岡2区,"POLYGON ((130.39449 33.60240, 130.39452 33.602..."


### 選挙区ごとにディソルブ


In [3]:
%%time
gdf_dissolved = gdf_original.dissolve(by="kucode")
assert gdf_dissolved.shape[0] == 289
gdf_dissolved

CPU times: user 34.6 s, sys: 2.19 s, total: 36.8 s
Wall time: 37 s


,geometry,UserID,ken,ku,kuname
kucode,,,,,
101,"POLYGON ((141.09830 43.11764, 141.09985 43.116...",7957,1,1,北海道1区
102,"POLYGON ((141.44707 43.15616, 141.44694 43.155...",7958,1,2,北海道2区
103,"POLYGON ((141.45414 43.01817, 141.45418 43.018...",7959,1,3,北海道3区
104,"MULTIPOLYGON (((139.83899 42.62443, 139.83895 ...",141,1,4,北海道4区
105,"POLYGON ((141.77799 42.93728, 141.77800 42.937...",7960,1,5,北海道5区
...,...,...,...,...,...
4604,"MULTIPOLYGON (((130.46696 30.23334, 130.46692 ...",105679,46,4,鹿児島4区
4701,"MULTIPOLYGON (((126.82083 26.26920, 126.82087 ...",114102,47,1,沖縄1区
4702,"MULTIPOLYGON (((127.68167 26.24366, 127.68143 ...",115323,47,2,沖縄2区


### 面積を算出

`EPSG:6677` - 平面直角座標系 9 系（東京都など）

厳密には対象地域ごとに適切な”系”を適用すべきだが、今回は簡単のため 9 系を全国に対して利用した。

参考: [平面直角座標系（平成十四年国土交通省告示第九号） | 国土地理院](https://www.gsi.go.jp/LAW/heimencho.html#9)


In [4]:
gdf_dissolved.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
gdf_dissolved = gdf_dissolved.to_crs(epsg=6677)
gdf_dissolved.crs

<Projected CRS: EPSG:6677>
Name: JGD2011 / Japan Plane Rectangular CS IX
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Japan - onshore - Honshu - Tokyo-to. (Excludes offshore island areas of Tokyo-to covered by Japan Plane Rectangular Coordinate System zones XIV, XVIII and XIX).
- bounds: (138.4, 29.31, 141.11, 37.98)
Coordinate Operation:
- name: Japan Plane Rectangular CS zone IX
- method: Transverse Mercator
Datum: Japanese Geodetic Datum 2011
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
gdf_dissolved["area_sqkm"] = gdf_dissolved.geometry.area / 1e6
gdf_dissolved.sort_values("area_sqkm", ascending=False)

,geometry,UserID,ken,ku,kuname,area_sqkm
kucode,,,,,,
112,"MULTIPOLYGON (((88378.525 1048671.974, 88380.3...",945,1,12,北海道12区,15342.919529
111,"MULTIPOLYGON (((287055.660 698491.775, 287054....",7429,1,11,北海道11区,10850.557654
106,"POLYGON ((230886.832 824969.267, 230898.899 82...",7970,1,6,北海道6区,10628.661669
302,"MULTIPOLYGON (((162881.700 327529.105, 162882....",10236,3,2,岩手2区,9654.498205
107,"MULTIPOLYGON (((398543.978 781192.476, 398497....",901,1,7,北海道7区,9528.605895
...,...,...,...,...,...,...
1310,"POLYGON ((-10826.396 -28250.712, -10812.557 -2...",38469,13,10,東京10区,24.293394
1311,"POLYGON ((-12768.888 -23963.291, -12735.445 -2...",38484,13,11,東京11区,23.384299
1327,"POLYGON ((-18213.536 -29339.510, -18159.829 -2...",38483,13,27,東京27区,21.844190


## 統計


In [7]:
df = pd.read_csv(
    "./source/senkyoku2022_toukei/02_人口総数_外国人人口_世帯数.csv",
    encoding="shift-jis",
)
assert df.shape[0] == 289
df

,ken,kuno,kucode,kuname,人口_総数,人口_男,人口_女,外国人人口,世帯数
0,1,1,101,北海道1区,527598,241025,286573,3939,273961
1,1,2,102,北海道2区,546641,258666,287975,4015,265360
2,1,3,103,北海道3区,530444,248142,282302,3057,265120
3,1,4,104,北海道4区,480342,225478,254864,3489,217789
4,1,5,105,北海道5区,510595,244132,266463,2990,225720
...,...,...,...,...,...,...,...,...,...
284,46,4,4604,鹿児島4区,386513,184574,201939,2949,177412
285,47,1,4701,沖縄1区,329331,160439,168892,4671,150191
286,47,2,4702,沖縄2区,373853,182439,191414,5546,153102
287,47,3,4703,沖縄3区,396314,196962,199352,5065,164773


## マージ


In [8]:
gdf_merged = gdf_dissolved.merge(df[["kucode", "人口_総数"]], on="kucode")
gdf_merged.columns = [
    "kucode",
    "geometry",
    "UserID",
    "ken",
    "ku",
    "kuname",
    "area_sqkm",
    "population",
]
gdf_merged = gdf_merged.drop(columns="UserID")
gdf_merged

,kucode,geometry,ken,ku,kuname,area_sqkm,population
0,101,"POLYGON ((102939.722 790943.841, 103067.759 79...",1,1,北海道1区,770.183366,527598
1,102,"POLYGON ((131239.968 795710.586, 131229.998 79...",1,2,北海道2区,118.637089,546641
2,103,"POLYGON ((132111.201 780393.314, 132114.909 78...",1,3,北海道3区,135.646770,530444
3,104,"MULTIPOLYGON (((463.981 735382.328, 460.403 73...",1,4,北海道4区,5095.657502,480342
4,105,"POLYGON ((158716.331 771968.134, 158718.099 77...",1,5,北海道5区,1726.264981,510595
...,...,...,...,...,...,...,...
284,4604,"MULTIPOLYGON (((-903506.011 -602103.465, -9035...",46,4,鹿児島4区,3767.056324,386513
285,4701,"MULTIPOLYGON (((-1306524.222 -1012356.490, -13...",47,1,沖縄1区,203.033046,329331
286,4702,"MULTIPOLYGON (((-1219544.112 -1023845.707, -12...",47,2,沖縄2区,151.863331,373853
287,4703,"MULTIPOLYGON (((-1203043.117 -1016396.491, -12...",47,3,沖縄3区,995.630480,396314


In [9]:
gdf_merged.to_file("districts.gpkg")

# 出力


In [10]:
gdf_merged = gpd.read_file("districts.gpkg")

### 極小ポリゴンを除去


In [11]:
def remove_small_parts(geometry, min_area=1e6):
    if isinstance(geometry, MultiPolygon):
        filtered_polygons = [
            polygon for polygon in geometry.geoms if polygon.area >= min_area
        ]
        if len(filtered_polygons) > 0:
            return MultiPolygon(filtered_polygons)
        else:
            return None
    return geometry

In [12]:
gdf_merged["geometry"] = gdf_merged["geometry"].apply(
    lambda geom: remove_small_parts(geom, min_area=1e6)
)

### ポリゴンを簡略化


In [13]:
%%time
gdf_merged["geometry"] = gdf_merged["geometry"].simplify(50)

CPU times: user 7.26 s, sys: 291 ms, total: 7.55 s
Wall time: 7.62 s


### GeoJSON ファイルへ


In [14]:
gdf_merged.to_crs(epsg=4326).to_file("districts.geojson", driver="GeoJSON")

### TopoJSON ファイルへ


In [15]:
from topojson import Topology

In [16]:
topo = Topology(gdf_merged)
topo.to_json("districts.topojson")